In [ ]:
# Wordle AI Solver Notebook

# This notebook runs the Wordle AI logic from `main.py`.

# **Important:** This AI is designed for **5-letter words** based on the provided code and word lists.

# **Setup:** Make sure this notebook (`Test1.ipynb`) is located in the same directory as `main.py`, `words.txt`, and `wordsAllowed.txt`.

In [ ]:
## Mode 1: Automatic AI vs. Random Word

# This mode simulates the AI playing against a randomly chosen target word from `words.txt`. It uses the `test_highestFrequency` function from `main.py` (which we import as `gameEngine`).

# Change the number inside `test_highestFrequency()` to run more simulations (e.g., `test_highestFrequency(10)`).

In [3]:
import main as gameEngine
import random

# Reload the word lists in case they were modified by other cells/runs
gameEngine.available_words = [i[:-1] for i in open("words.txt", "r").readlines()]
gameEngine.permanent_answers = [i[:-1] for i in open("words.txt", "r").readlines()]
gameEngine.wordsAllowed = [i[:-1] for i in open("wordsAllowed.txt", "r").readlines()]

print("Running 1 automatic game simulation...")
# The test_highestFrequency function already prints its results
gameEngine.test_highestFrequency(1)

print("\nSimulation complete.")

Running 1 automatic game simulation...

--- Starting Verbose Test Run (1 games) ---

--- Game 1 ---
Target Word: tumor
Guess 1: salet -> BBBBY
  Remaining possible words: 86
Guess 2: north -> BYYYB
  Remaining possible words: 3
  (Blimp condition detected)
Guess 3: dumps -> BGGBB
  Remaining possible words: 1
Guess 4: tumor -> GGGGG
Solved in 4 steps!
Guess History for game 1: ['salet', 'north', 'dumps', 'tumor']

--- Overall Results ---
Method: highest_freq
Total Games Simulated: 1
Results (Steps: Count): {'1': 0, '2': 0, '3': 0, '4': 1, '5': 0, '6': 0, 'DNF': 0}
Success Rate: 1.0000
Average Steps (for successful games): 4.0000

Simulation complete.


In [ ]:
# ## Mode 2: AI vs. User-Defined Word

# In this mode, you provide a specific 5-letter target word (which must be a valid potential answer from `words.txt`), and the AI will attempt to guess it.

# We'll define a helper function `solve_specific_word` based on the logic in `test_highestFrequency`.

In [13]:
import main as gameEngine
import random
import math # Needed for isBlimp logic

# --- Ensure this function is defined either in main.py or in a cell ABOVE this one ---
def get_guess_colors(guess, target_word):
    """
    Compares a guess to the target word and returns a string representing
    Wordle colors (G=Green, Y=Yellow, B=Black/Gray).
    Handles duplicate letters correctly.
    """
    if len(guess) != 5 or len(target_word) != 5:
        return "Error" # Should not happen

    colors = [''] * 5  # Initialize with placeholders
    target_list = list(target_word) # Mutable list to track used letters

    # First pass for Greens (Exact Matches)
    for i in range(5):
        if guess[i] == target_list[i]:
            colors[i] = 'G'
            target_list[i] = None # Mark this letter as used for green

    # Second pass for Yellows (Present but Wrong Position)
    for i in range(5):
        if colors[i] == '': # Only check letters not already marked Green
            if guess[i] in target_list:
                colors[i] = 'Y'
                # Mark the *first* available instance of this letter in target as used
                target_list[target_list.index(guess[i])] = None
            else:
                # If not Green or Yellow, it must be Black/Gray
                colors[i] = 'B'

    return "".join(colors) # Return as a single string like "BGYBB"
# --- End of get_guess_colors function ---


# --- MODIFIED solve_specific_word function ---
def solve_specific_word(target_word):
    target_word = target_word.lower() # Ensure lowercase
    # --- Validation ---
    if len(target_word) != 5:
        print(f"Error: '{target_word}' is not 5 letters long.")
        return

    # Reload lists each time to ensure clean state
    # Assuming main.py is imported as gameEngine
    initial_word_list = [i[:-1] for i in open("words.txt", "r").readlines()]
    allowed_guesses = [i[:-1] for i in open("wordsAllowed.txt", "r").readlines()]

    if target_word not in initial_word_list:
        print(f"Error: '{target_word}' is not in the list of possible answers (words.txt).")
        return

    # --- Simulation Logic ---
    available_words = initial_word_list[:]
    steps = 0
    guess_history = []

    print(f"Target Word: {target_word}")

    # --- First Guess ---
    guess = "salet"
    steps = 1
    guess_history.append(guess)
    colors = get_guess_colors(guess, target_word) # Get colors
    print(f"Guess {steps}: {guess} -> {colors}") # Print with colors

    if guess == target_word:
        print(f"Solved in {steps} steps!")
        print(f"History: {guess_history}")
        return

    available_words = gameEngine.filter_words(available_words, guess, target_word)
    print(f"  Remaining possible words: {len(available_words)}")
    if target_word not in available_words and len(available_words) > 0:
         print(f"Warning: Target word '{target_word}' was filtered out! List: {available_words[:10]}")

    # --- Subsequent Guesses (Loop) ---
    for j in range(5): # Max 5 more guesses
        if not available_words:
             print("Error: No possible words left in list.")
             steps = -1 # Indicate failure state
             break

        if len(available_words) == 1:
            guess = available_words[0]
        elif gameEngine.isBlimp(available_words):
            print("  (Blimp condition detected)")
            guess = gameEngine.blimpSearch(available_words)
        else:
            guess = gameEngine.getMaxValue1(available_words)

        steps += 1
        guess_history.append(guess)
        colors = get_guess_colors(guess, target_word) # Get colors
        print(f"Guess {steps}: {guess} -> {colors}") # Print with colors

        if guess == target_word:
            print(f"Solved in {steps} steps!")
            print(f"History: {guess_history}")
            return

        available_words = gameEngine.filter_words(available_words, guess, target_word)
        print(f"  Remaining possible words: {len(available_words)}")
        if target_word not in available_words and len(available_words) > 0:
            print(f"Warning: Target word '{target_word}' was filtered out! List: {available_words[:10]}")

    # --- End of Loop ---
    if steps != -1 and guess != target_word:
        print(f"Failed to solve in 6 steps.")
        print(f"History: {guess_history}")
        print(f"Remaining possibilities: {available_words}")

# --- Get User Input and Run ---
user_target = input("Enter the 5-letter target word (must be in words.txt): ")
solve_specific_word(user_target)

Enter the 5-letter target word (must be in words.txt):  house


Target Word: house
Guess 1: salet -> YBBYB
  Remaining possible words: 38
Guess 2: rouse -> BGGGG
  Remaining possible words: 2
Guess 3: house -> GGGGG
Solved in 3 steps!
History: ['salet', 'rouse', 'house']
